In [1]:
import sagemaker
import numpy as np
import torch
import torchvision
import os
import json
import boto3
from sagemaker.tuner import CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.pytorch import PyTorch

/home/tho121/udacity_aws/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sagemaker import get_execution_role

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='SagemakerFullAccess')['Role']['Arn']

Couldn't call 'get_role' to get Role ARN from role name Tony to get Role path.


In [65]:
from sagemaker.huggingface import HuggingFace

#os.environ['SM_CHANNEL_TRAIN']='s3://awscapstone/train_data'
#os.environ['SM_CHANNEL_TEST']='s3://awscapstone/test_data'

train_dir = 's3://awscapstone/train_data'
test_dir = 's3://awscapstone/test_data'
#cache_dir = 's3://awscapstone/cached_data'
#os.environ['SM_MODEL_DIR']='s3://awscapstone'

# hyperparameters, which are passed into the training job
hyperparameters={'num_train_epochs': 10, # train epochs
                 'learning_rate': 2e-5,
                 'per_device_train_batch_size': 64, # batch size
                 'per_device_eval_batch_size': 64, # batch size
                 'dataloader_workers': 4,
                 'model_name': "google/vit-base-patch16-224-in21k", # model which will be trained on
                 #"training_dir": train_dir, 
                 #"test_dir": test_dir
                 }

In [46]:
distribution = {'smdistributed':{'dataparallel':{ 'enabled': True }}}

In [54]:
import uuid
bucket = "awscapstone"#sagemaker.Session().default_bucket()
prefix = 'vit'

checkpoint_suffix = str(uuid.uuid4())[:8]
checkpoint_s3_path = 's3://{}/checkpoint-{}-{}'.format(bucket,prefix,checkpoint_suffix)
checkpoint_local_path='/opt/ml/checkpoints'

print('Checkpointing Path: {}'.format(checkpoint_s3_path))

use_spot_instances = True
max_run = 60*60*5    #1+ hour 
max_wait = 60*60*24 if use_spot_instances else None

Checkpointing Path: s3://awscapstone/checkpoint-vit-77e25880


In [66]:


vit_estimator = HuggingFace(entry_point='train_vit_hf.py',
                    base_job_name='capstone-vit',
                    instance_type='ml.p3.2xlarge',
                    instance_count=1,
                    role=role,
                    #transformers_version='4.17',
                    #pytorch_version='1.10',
                    py_version='py38',
                    #distribution=distribution,
                    image_uri='149633813601.dkr.ecr.us-east-1.amazonaws.com/dockercontainers:latest',
                    hyperparameters=hyperparameters,
                    #use_spot_instances=use_spot_instances,
                    #max_run=max_run,
                    #max_wait=max_wait,
                    #checkpoint_s3_uri=checkpoint_s3_path
                    )



In [68]:

vit_estimator.fit({"train": train_dir, "test": test_dir}, wait=True)

INFO:sagemaker:Creating training-job with name: capstone-vit-2023-01-07-02-17-24-071


KeyboardInterrupt: 